In [4]:
print('Hola')

Hola


In [1]:
import cv2

In [2]:
import torch 
print(torch.__version__)

2.6.0+cu118


In [3]:
import torch
print(torch.cuda.is_available())  # True si se detecta la GPU


True


In [9]:
import os, random

carpeta = "data/train/publaynet/train"
imagenes = random.sample(os.listdir(carpeta), 3)

for img in imagenes:
    cv2.imshow(img, cv2.imread(os.path.join(carpeta, img)))

cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import json

with open("data/labels/publaynet/train.json", "r") as f:
    data = json.load(f)

sample = {
    "images": data["images"][:2],  # Solo 2 imágenes
    "annotations": data["annotations"][:5],  # Solo 5 anotaciones
    "categories": data["categories"]  # Si existe
}

with open("sample.json", "w") as f:
    json.dump(sample, f, indent=4)


In [10]:
import json
import cv2
import os

# Cargar el archivo JSON con las anotaciones
with open('sample.json', 'r') as f:  # Cambia la ruta según corresponda
    data = json.load(f)

# Crear un directorio para guardar las imágenes recortadas si no existe
output_dir = 'recortes_texto'
os.makedirs(output_dir, exist_ok=True)

# Iterar sobre las anotaciones
for annotation in data['annotations']:
    # Obtener el nombre del archivo de la imagen directamente
    image_name = next(item for item in data['images'] if item['id'] == annotation['image_id'])['file_name']
    
    # Cargar la imagen
    image = cv2.imread(f'data/train/publaynet/train/{image_name}')  # Cambia la ruta a donde están tus imágenes
    
    # Obtener las coordenadas del cuadro delimitador (bbox)
    x, y, w, h = annotation['bbox']
    
    # Recortar la región de la imagen
    cropped_image = image[int(y):int(y+h), int(x):int(x+w)]
    
    # Obtener el nombre de la categoría
    category_id = annotation['category_id']
    category_name = next(item for item in data['categories'] if item['id'] == category_id)['name']
    
    # Crear un nombre de archivo para la imagen recortada
    cropped_image_name = f"{category_name}_{image_name.split('.')[0]}_id{annotation['id']}.jpg"
    
    # Guardar la imagen recortada
    cv2.imwrite(os.path.join(output_dir, cropped_image_name), cropped_image)
    print(f"Imagen recortada guardada como: {cropped_image_name}")

print("Proceso de recorte completado.")


TypeError: 'NoneType' object is not subscriptable

In [7]:
import json
import random

# Cargar el archivo original train.json
with open("data/labels/publaynet/train.json", 'r') as f:
    train_data = json.load(f)

# Seleccionar aleatoriamente 100 imágenes
selected_images = random.sample(train_data['images'], 100)

# Crear un set de los 'image_id' de las imágenes seleccionadas
selected_image_ids = {image['id'] for image in selected_images}

# Filtrar las anotaciones para que solo incluyan aquellas cuyo 'image_id' esté en selected_image_ids
selected_annotations = [annotation for annotation in train_data['annotations'] if annotation['image_id'] in selected_image_ids]

# Crear el nuevo diccionario para el sample.json
sample_data = {
    "images": selected_images,
    "annotations": selected_annotations,
    "categories": train_data['categories']  # Mantener las categorías igual
}

# Guardar el nuevo archivo sample.json
with open('sample.json', 'w') as f:
    json.dump(sample_data, f, indent=4)

print("Archivo sample.json generado con 100 imágenes.")


Archivo sample.json generado con 100 imágenes.


In [8]:
import os
import cv2
import json

# Crear las carpetas para las anotaciones
output_dir = 'recortes'
categories = ['text', 'title', 'list', 'table', 'figure']

# Crear la carpeta principal de recortes si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Crear subcarpetas para cada tipo de anotación
for category in categories:
    category_path = os.path.join(output_dir, category)
    if not os.path.exists(category_path):
        os.makedirs(category_path)

# Cargar el archivo sample.json
with open('sample.json', 'r') as f:
    sample_data = json.load(f)

# Función para recortar la imagen según las coordenadas de bbox
def crop_image(image_path, bbox, output_path):
    image = cv2.imread(image_path)
    x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    cropped_image = image[y:y+h, x:x+w]  # Recortar la imagen según el bbox
    cv2.imwrite(output_path, cropped_image)  # Guardar la imagen recortada

# Procesar cada anotación
for annotation in sample_data['annotations']:
    image_id = annotation['image_id']
    category_id = annotation['category_id']
    category_name = sample_data['categories'][category_id - 1]['name']  # Obtener el nombre de la categoría

    # Buscar la imagen correspondiente en el archivo
    image_info = next(image for image in sample_data['images'] if image['id'] == image_id)
    image_name = image_info['file_name']
    image_path = os.path.join('data/train/publaynet/train', image_name)  # Ruta a la imagen original
    
    # Definir el directorio de destino según el tipo de anotación
    category_folder = os.path.join(output_dir, category_name)
    
    # Definir el nombre del archivo de salida
    output_image_name = f"{image_name.split('.')[0]}_{annotation['id']}.jpg"
    output_image_path = os.path.join(category_folder, output_image_name)
    
    # Recortar y guardar la imagen
    crop_image(image_path, annotation['bbox'], output_image_path)

print("Recortes de imágenes generados exitosamente.")


TypeError: 'NoneType' object is not subscriptable

In [9]:
import os
import cv2
import json

# Crear las carpetas para las anotaciones
output_dir = 'recortes'
categories = ['text', 'title', 'list', 'table', 'figure']

# Crear la carpeta principal de recortes si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Crear subcarpetas para cada tipo de anotación
for category in categories:
    category_path = os.path.join(output_dir, category)
    if not os.path.exists(category_path):
        os.makedirs(category_path)

# Cargar el archivo sample.json
with open('sample.json', 'r') as f:
    data = json.load(f)

# Iterar sobre las anotaciones
for annotation in data['annotations']:
    # Obtener el nombre del archivo de la imagen directamente
    image_name = next(item for item in data['images'] if item['id'] == annotation['image_id'])['file_name']
    
    # Cargar la imagen
    image_path = os.path.join('data/train/publaynet/train', image_name)  # Cambia la ruta a donde están tus imágenes
    image = cv2.imread(image_path)
    
    # Obtener las coordenadas del cuadro delimitador (bbox)
    x, y, w, h = annotation['bbox']
    
    # Recortar la región de la imagen
    cropped_image = image[int(y):int(y+h), int(x):int(x+w)]
    
    # Obtener el nombre de la categoría
    category_id = annotation['category_id']
    category_name = next(item for item in data['categories'] if item['id'] == category_id)['name']
    
    # Crear un nombre de archivo para la imagen recortada
    cropped_image_name = f"{category_name}_{image_name.split('.')[0]}_id{annotation['id']}.jpg"
    
    # Crear el directorio para la categoría si no existe
    category_folder = os.path.join(output_dir, category_name)
    if not os.path.exists(category_folder):
        os.makedirs(category_folder)
    
    # Guardar la imagen recortada
    cv2.imwrite(os.path.join(category_folder, cropped_image_name), cropped_image)
    print(f"Imagen recortada guardada como: {os.path.join(category_folder, cropped_image_name)}")

print("Proceso de recorte completado.")


TypeError: 'NoneType' object is not subscriptable

In [11]:
import os
import cv2
import json

# Ruta de las imágenes
image_base_path = 'data/train/publaynet/train'  # Cambiar si es necesario

# Ruta del archivo JSON de anotaciones
annotations_file = 'sample.json'

# Crear la carpeta principal de recortes si no existe
output_dir = 'recortes'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Cargar el archivo sample.json
with open(annotations_file, 'r') as f:
    data = json.load(f)

# Iterar sobre las anotaciones
for annotation in data['annotations']:
    # Obtener el nombre del archivo de la imagen directamente
    image_info = next(item for item in data['images'] if item['id'] == annotation['image_id'])
    image_name = image_info['file_name']
    
    # Ruta completa de la imagen
    image_path = os.path.join(image_base_path, image_name)
    
    # Verificar si la imagen existe
    if not os.path.exists(image_path):
        print(f"Advertencia: La imagen {image_name} no se encuentra en la ruta especificada.")
        continue
    
    # Cargar la imagen
    image = cv2.imread(image_path)
    
    # Obtener las coordenadas del cuadro delimitador (bbox)
    x, y, w, h = annotation['bbox']
    
    # Recortar la región de la imagen
    cropped_image = image[int(y):int(y+h), int(x):int(x+w)]
    
    # Obtener el nombre de la categoría
    category_id = annotation['category_id']
    category_name = next(item for item in data['categories'] if item['id'] == category_id)['name']
    
    # Crear el directorio para la categoría si no existe
    category_folder = os.path.join(output_dir, category_name)
    if not os.path.exists(category_folder):
        os.makedirs(category_folder)
    
    # Crear un nombre de archivo para la imagen recortada
    cropped_image_name = f"{category_name}_{image_name.split('.')[0]}_id{annotation['id']}.jpg"
    
    # Guardar la imagen recortada
    cropped_image_path = os.path.join(category_folder, cropped_image_name)
    cv2.imwrite(cropped_image_path, cropped_image)
    
    print(f"Imagen recortada guardada como: {cropped_image_path}")

print("Proceso de recorte completado.")


Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la ruta especificada.
Advertencia: La imagen PMC3441149_00037.jpg no se encuentra en la

In [13]:
import os
import json
import random

# Ruta del archivo JSON original (train.json)
train_json_path = 'data/labels/publaynet/train.json'

# Ruta de las imágenes disponibles
image_base_path = 'data/train/publaynet/train'

# Ruta donde guardar el nuevo sample.json
sample_json_path = 'sample.json'

# Cargar el archivo train.json
with open(train_json_path, 'r') as f:
    train_data = json.load(f)

# Obtener el listado de imágenes disponibles en la carpeta
available_images = set(os.listdir(image_base_path))

# Filtrar las imágenes que existen en la carpeta
valid_images = [image for image in train_data['images'] if image['file_name'] in available_images]

# Seleccionar aleatoriamente 100 imágenes
selected_images = random.sample(valid_images, 100)

# Crear un conjunto de ID de imágenes seleccionadas para facilitar la búsqueda de anotaciones
selected_image_ids = {image['id'] for image in selected_images}

# Filtrar las anotaciones para que solo incluya aquellas que corresponden a las imágenes seleccionadas
selected_annotations = [annotation for annotation in train_data['annotations'] if annotation['image_id'] in selected_image_ids]

# Crear el nuevo archivo sample.json con las imágenes y anotaciones seleccionadas
sample_data = {
    'images': selected_images,
    'annotations': selected_annotations,
    'categories': train_data['categories']  # Las categorías no cambian, las mantenemos
}

# Guardar el sample.json
with open(sample_json_path, 'w') as f:
    json.dump(sample_data, f, indent=4)

print("El archivo sample.json ha sido generado exitosamente con 100 imágenes.")


El archivo sample.json ha sido generado exitosamente con 100 imágenes.


In [1]:
import os
import cv2
import json
import shutil

# Directorio de salida
output_dir = 'recortes'
categories = ['text', 'title', 'list', 'table', 'figure']

# Eliminar la carpeta de recortes si existe y volver a crearla
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# Crear subcarpetas para cada tipo de anotación
for category in categories:
    os.makedirs(os.path.join(output_dir, category))

# Cargar el archivo sample.json
with open('sample.json', 'r') as f:
    sample_data = json.load(f)

def crop_image(image_path, bbox, output_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Advertencia: No se pudo cargar la imagen {image_path}")
        return

    height, width = image.shape[:2]
    x, y, w, h = map(int, bbox)

    # Añadir 5 píxeles de margen en cada dirección
    x1 = max(x - 5, 0)
    y1 = max(y - 5, 0)
    x2 = min(x + w + 5, width)
    y2 = min(y + h + 5, height)

    cropped_image = image[y1:y2, x1:x2]
        
    # Escalar al 200% (2x en cada eje)
    scaled_image = cv2.resize(cropped_image, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)

    # Guardar la imagen escalada
    cv2.imwrite(output_path, scaled_image)


# Procesar cada anotación
for annotation in sample_data['annotations']:
    image_id = annotation['image_id']
    category_id = annotation['category_id']
    
    # Obtener el nombre de la categoría
    category_name = next(item for item in sample_data['categories'] if item['id'] == category_id)['name']
    
    # Buscar la imagen correspondiente
    image_info = next(image for image in sample_data['images'] if image['id'] == image_id)
    image_name = image_info['file_name']
    image_path = os.path.join('data/train/publaynet/train', image_name)  # Ruta de la imagen original
    
    # Definir el directorio de destino según el tipo de anotación
    category_folder = os.path.join(output_dir, category_name)
    
    # Definir el nombre del archivo de salida
    output_image_name = f"{image_name.split('.')[0]}_{annotation['id']}.jpg"
    output_image_path = os.path.join(category_folder, output_image_name)
    
    # Recortar y guardar la imagen
    crop_image(image_path, annotation['bbox'], output_image_path)

print("Recortes de imágenes generados exitosamente.")


Recortes de imágenes generados exitosamente.
